In [ ]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_contrast(image, lower=0.2, upper=2.0)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=0.2, upper = 0.2)
    image = tf.image.random_hue(image, max_delta = 0.08)
    image = tf.image.random_saturation(image, lower=0.2, upper=0.2)
    return image, label
    

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax', bias_regularizer=tf.keras.regularizers.l2(0.01))
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        2

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(3).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [9]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [10]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [11]:
history = model.fit(train_data,
                    epochs=25,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 11:14:38.934786: {'loss': 3.8386325672149657, 'accuracy': 0.3284, 'val_loss': 1.6191903588118826, 'val_accuracy': 0.4179}
Epoch 1 ends at 11:18:51.720424: {'loss': 1.5236468575795492, 'accuracy': 0.45151666, 'val_loss': 1.540223212758447, 'val_accuracy': 0.4487}
Epoch 2 ends at 11:23:12.705720: {'loss': 1.5465578357378642, 'accuracy': 0.48793334, 'val_loss': 1.3888446229278661, 'val_accuracy': 0.4987}
Epoch 3 ends at 11:27:32.299920: {'loss': 1.2038467154184977, 'accuracy': 0.5684417, 'val_loss': 1.1033646589631487, 'val_accuracy': 0.6049}
Epoch 4 ends at 11:31:57.611306: {'loss': 0.977924930826823, 'accuracy': 0.65434164, 'val_loss': 0.8626701687551608, 'val_accuracy': 0.6989}
Epoch 5 ends at 11:36:24.343681: {'loss': 0.818302646970749, 'accuracy': 0.7126667, 'val_loss': 20.746122759618576, 'val_accuracy': 0.241}


Epoch 6 ends at 11:40:22.037226: {'loss': 0.7334151373212419, 'accuracy': 0.7412948}


KeyboardInterrupt: 

In [ ]:
model.evaluate(test_data)

In [ ]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))